# bns-nlp-engine Hızlı Başlangıç

Bu notebook, bns-nlp-engine kütüphanesinin temel özelliklerini gösterir.

## Kurulum

```bash
pip install bns-nlp-engine[all]
```

In [ ]:
# Import'lar
import asyncio
from bnsnlp import Pipeline, Config
from bnsnlp.core.registry import PluginRegistry
from bnsnlp.preprocess import TurkishPreprocessor
from bnsnlp.embed import OpenAIEmbedder, HuggingFaceEmbedder
from bnsnlp.search import FAISSSearch
from bnsnlp.classify import TurkishClassifier

## 1. Metin Ön İşleme

Türkçe metinleri normalize edin, tokenize edin ve temizleyin.

In [ ]:
# Preprocessor oluştur
preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': True,
    'lemmatize': True
})

# Tek metin işle
text = "Merhaba DÜNYA! Bu bir TEST metnidir."
result = await preprocessor.process(text)

print(f"Orijinal: {text}")
print(f"İşlenmiş: {result.text}")
print(f"Tokenlar: {result.tokens}")

In [ ]:
# Batch işleme
texts = [
    "Python programlama dili",
    "Makine öğrenmesi algoritmaları",
    "Doğal dil işleme teknikleri"
]

results = await preprocessor.process(texts)

for i, result in enumerate(results):
    print(f"\n{i+1}. Metin:")
    print(f"   İşlenmiş: {result.text}")
    print(f"   Token sayısı: {len(result.tokens)}")

## 2. Embedding Oluşturma

Metinleri vektörlere dönüştürün.

### OpenAI Embeddings

In [ ]:
import os

# API key'i environment variable'dan al
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    embedder = OpenAIEmbedder({
        'api_key': api_key,
        'model': 'text-embedding-3-small',
        'batch_size': 16
    })
    
    # Tek metin
    result = await embedder.embed("Merhaba dünya")
    print(f"Embedding boyutu: {result.dimensions}")
    print(f"İlk 5 değer: {result.embeddings[0][:5]}")
    
    # Batch
    result = await embedder.embed(texts)
    print(f"\nBatch: {len(result.embeddings)} embedding oluşturuldu")
else:
    print("OPENAI_API_KEY environment variable tanımlı değil")

### HuggingFace Embeddings (Yerel)

In [ ]:
# Yerel model kullan (API key gerekmez)
hf_embedder = HuggingFaceEmbedder({
    'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'use_gpu': False  # CPU kullan
})

result = await hf_embedder.embed("Merhaba dünya")
print(f"Embedding boyutu: {result.dimensions}")
print(f"Model: {result.model}")

## 3. Semantik Arama

FAISS ile yerel semantik arama.

In [ ]:
# FAISS search oluştur
search = FAISSSearch({'dimension': result.dimensions})

# Dökümanları hazırla
documents = [
    "Python programlama dili",
    "Makine öğrenmesi algoritmaları",
    "Doğal dil işleme teknikleri",
    "Veri bilimi ve analitik",
    "Derin öğrenme modelleri"
]

# Embedding'leri oluştur
doc_embeddings = await hf_embedder.embed(documents)

# İndeksle
ids = [str(i) for i in range(len(documents))]
metadata = [{'category': 'tech', 'index': i} for i in range(len(documents))]

await search.index(documents, doc_embeddings.embeddings, ids, metadata)

print(f"{len(documents)} döküman indekslendi")

In [ ]:
# Arama yap
query = "NLP teknikleri"
query_embedding = await hf_embedder.embed(query)

results = await search.search(
    query_embedding.embeddings[0],
    top_k=3
)

print(f"Query: {query}\n")
print(f"Sonuçlar ({results.query_time_ms:.2f}ms):\n")

for i, result in enumerate(results.results, 1):
    print(f"{i}. {result.text}")
    print(f"   Benzerlik: {result.score:.4f}")
    print(f"   Metadata: {result.metadata}\n")

## 4. Pipeline Kullanımı

Birden fazla adımı zincirleyin.

In [ ]:
# Registry ve config
registry = PluginRegistry()
registry.discover_plugins()
config = Config()

# Pipeline oluştur
pipeline = Pipeline(config, registry)

# Adımları ekle
pipeline.add_step('preprocess', 'turkish', config={
    'lowercase': True,
    'remove_punctuation': True,
    'lemmatize': True
})

# İşle
text = "Merhaba DÜNYA! Bu bir TEST metnidir."
result = await pipeline.process(text)

print(f"Pipeline sonucu: {result.text}")

## 5. Tam Örnek: End-to-End

Preprocessing, embedding ve search'ü birlikte kullanın.

In [ ]:
async def full_example():
    # 1. Dökümanları hazırla
    documents = [
        "Python güçlü bir programlama dilidir",
        "Makine öğrenmesi yapay zeka alt dalıdır",
        "Doğal dil işleme metinleri analiz eder",
        "Veri bilimi veriden içgörü çıkarır",
        "Derin öğrenme sinir ağları kullanır"
    ]
    
    # 2. Preprocessing
    preprocessor = TurkishPreprocessor({
        'lowercase': True,
        'remove_punctuation': True,
        'remove_stopwords': True
    })
    
    preprocessed = await preprocessor.process(documents)
    processed_texts = [p.text for p in preprocessed]
    
    # 3. Embedding
    embedder = HuggingFaceEmbedder({
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'use_gpu': False
    })
    
    embeddings = await embedder.embed(processed_texts)
    
    # 4. İndeksleme
    search = FAISSSearch({'dimension': embeddings.dimensions})
    ids = [str(i) for i in range(len(documents))]
    
    await search.index(
        documents,  # Orijinal metinleri sakla
        embeddings.embeddings,
        ids
    )
    
    # 5. Arama
    query = "yapay zeka ve makine öğrenmesi"
    
    # Query'yi işle
    query_preprocessed = await preprocessor.process(query)
    query_embedding = await embedder.embed(query_preprocessed.text)
    
    # Ara
    results = await search.search(
        query_embedding.embeddings[0],
        top_k=3
    )
    
    # Sonuçları göster
    print(f"Query: {query}\n")
    print("En benzer dökümanlar:\n")
    
    for i, result in enumerate(results.results, 1):
        print(f"{i}. {result.text}")
        print(f"   Benzerlik: {result.score:.4f}\n")

# Çalıştır
await full_example()

## Sonraki Adımlar

- [Preprocessing Notebook](preprocessing.ipynb) - Detaylı preprocessing örnekleri
- [Embeddings Notebook](embeddings.ipynb) - Farklı embedding provider'ları
- [Search Notebook](search.ipynb) - İleri seviye arama teknikleri
- [Dokümantasyon](https://yourusername.github.io/bns-nlp-engine)